In [1]:
import numpy as np
import cv2
import glob
from matplotlib import pyplot as plt
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
img_color = cv2.imread('test_imgs/color_align.png')
img_depth = cv2.imread('test_imgs/depth_align.png', 0)
print('color shape: ', img_color.shape)
print('depth shape: ', img_depth.shape)

color shape:  (480, 640, 3)
depth shape:  (480, 640)


In [3]:
height, width = img_color.shape[:2] # 이미지의 높이와 너비 불러옴, 가로 [0], 세로[1]

img_hsv = cv2.cvtColor(img_color, cv2.COLOR_BGR2HSV) # cvtColor 함수를 이용하여 hsv 색공간으로 변환

In [4]:
colors = ['green', 'pink', 'yellow', 'blue', 'violet', 'red']

In [5]:
# RED
lower_red1 = np.array([0, 150, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([160,150,100])
upper_red2 = np.array([179,255,255])

# PINK
lower_pink1 = np.array([0, 30, 80])
upper_pink1 = np.array([10, 130, 255])
lower_pink2 = np.array([150,30,80])
upper_pink2 = np.array([179,130,255])

# GREEN
lower_green = (60-15, 30, 30)
upper_green = (60+15, 255, 255)

# YELLOW
lower_yellow = (30-10, 80, 80)
upper_yellow = (30+10, 255, 255)

# BLUE
lower_blue = (100-10, 30, 30)
upper_blue = (100+10, 255, 255)

# VIOLET
lower_violet = (135-20, 30, 30)
upper_violet = (135+20, 255, 255)

In [6]:
blocks_rgb_by_color = []
blocks_mask_by_color = []
for color in colors:
    if color == 'pink' or color =='red':
        for i in (1,2):
            exec(f"lower_color{i} = lower_{color}{i}")
            exec(f"upper_color{i} = upper_{color}{i}")

        mask_color1 = cv2.inRange(img_hsv, lower_color1, upper_color1)
        mask_color2 = cv2.inRange(img_hsv, lower_color2, upper_color2)
        img_mask_color = mask_color1 + mask_color2

        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        erosion_image_color = cv2.erode(img_mask_color, kernel, iterations=2)  #// make erosion image
        img_mask_color = cv2.dilate(erosion_image_color, kernel, iterations=2)  #// make dilation image

        # 바이너리 이미지를 마스크로 사용하여 원본이미지에서 범위값에 해당하는 영상부분을 획득
        img_result_color = cv2.bitwise_and(img_color, img_color, mask = img_mask_color) 
        
        exec(f"img_result_{color} = img_result_color")
    
    else:
        exec(f"lower_color = lower_{color}")
        exec(f"upper_color = upper_{color}")

        img_mask_color = cv2.inRange(img_hsv, lower_color, upper_color) # 범위내의 픽셀들은 흰색, 나머지 검은색

        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        erosion_image_color = cv2.erode(img_mask_color, kernel, iterations=2)  #// make erosion image
        img_mask_color = cv2.dilate(erosion_image_color, kernel, iterations=2)  #// make dilation image

        # 바이너리 이미지를 마스크로 사용하여 원본이미지에서 범위값에 해당하는 영상부분을 획득
        img_result_color = cv2.bitwise_and(img_color, img_color, mask = img_mask_color) 

        exec(f"img_result_{color} = img_result_color")
        
    _, src_bin = cv2.threshold(img_mask_color, 0, 255, cv2.THRESH_OTSU)

    cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(src_bin)

    blocks_color = []
    blocks_mask = []

    for i in range(1, cnt): # 각각의 객체 정보에 들어가기 위해 반복문. 범위를 1부터 시작한 이유는 배경을 제외
        (x, y, w, h, area) = stats[i]
        cen_x, cen_y = map(int, centroids[i])
        block_mask = (labels==i)*img_mask_color
        block_color = cv2.bitwise_and(img_color, img_color, mask = block_mask) 

        # 노이즈 제거
        if area < 500:
            continue
        
        blocks_color.append(block_color)
        blocks_mask.append(block_mask)
        
        
    exec(f"blocks_rgb_{color} = blocks_color")
    exec(f"blocks_mask_{color} = blocks_mask")
    exec(f"blocks_rgb_by_color.append(blocks_rgb_{color})")
    exec(f"blocks_mask_by_color.append(blocks_mask_{color})")
    

In [7]:
tower_mask = 0
tower_color = 0
for mask, color in zip(blocks_mask_by_color, blocks_rgb_by_color):
    for block_m in mask:

        tower_mask += block_m

        # cv2.imshow('blk mask', block_m)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
    
    for block_c in color:

        tower_color += block_c

        # cv2.imshow('blk color', block_c)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

In [8]:
cv2.imshow('tower mask', tower_mask)
cv2.imshow('tower color', tower_color)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
for c, b in zip(colors, blocks_mask_by_color):
    print(c)
    print(len(b))

green
7
pink
6
yellow
6
blue
9
violet
6
red
6


In [10]:
# temp intrinsic matrix
intrinsic = o3d.camera.PinholeCameraIntrinsic()
intrinsic.intrinsic_matrix = [[322.282420, 0, 320.818268],[0, 322.282420, 178.779297],[0, 0, 1]]

In [11]:
def get_pointcloud_from_color_depth(color_image, depth_image, intrinsic):
    o3d_img = o3d.geometry.Image()
    
    if isinstance(color_image, type(o3d_img)):
        pass
    elif isinstance(color_image, np.ndarray):
        color_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
        color_image = o3d.geometry.Image(color_image)
        
    if isinstance(depth_image, type(o3d_img)):
        pass
    elif isinstance(depth_image, np.ndarray):
        depth_image = o3d.geometry.Image(depth_image)
        
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_image, depth_image)
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, intrinsic)
    
    return pcd

In [12]:
pcd = get_pointcloud_from_color_depth(color_image=tower_color, depth_image=img_depth, intrinsic=intrinsic)

In [13]:
print(pcd)
print(np.asarray(pcd.points))
o3d.visualization.draw_geometries([pcd])

PointCloud with 255829 points.
[[-0.00099546 -0.00055473  0.001     ]
 [-0.00099235 -0.00055473  0.001     ]
 [-0.00098925 -0.00055473  0.001     ]
 ...
 [ 0.00026162  0.00024841  0.00026667]
 [ 0.00026245  0.00024841  0.00026667]
 [ 0.00026327  0.00024841  0.00026667]]


In [14]:
blocks_pcd_by_color = []
for color, block_mask in zip(colors, blocks_mask_by_color):
    print(color)
    blocks_pcd = []
    for msk in block_mask:
        masked_block_rgb = cv2.bitwise_and(tower_color, tower_color, mask = msk)
        masked_block_depth = cv2.bitwise_and(img_depth, img_depth, mask = msk)
        
        pcd = get_pointcloud_from_color_depth(color_image=masked_block_rgb, depth_image=masked_block_depth, intrinsic=intrinsic)
        blocks_pcd.append(pcd)
        
        o3d.visualization.draw_geometries([pcd])
    
    exec(f"blocks_pcd_{color} = blocks_pcd")
    exec(f"blocks_pcd_by_color.append(blocks_pcd_{color})")

green
pink
yellow
blue
violet
red


In [21]:
for color, block_pcd in zip(colors, blocks_pcd_by_color):
    print(color)
    for idx, p in enumerate(block_pcd):
        if color == 'green' and idx ==1:
            continue
        # print(p)
        print(np.asarray(p.points)[:,0].mean() * 1000000, '\t',np.asarray(p.points)[:,1].mean() * 1000000)

green
38.25654706613877 	 -78.64555935871452
35.54817713948268 	 -20.247265635719714
37.952101765413765 	 35.754529883540556
52.415320260938195 	 97.22881273638588
38.16944512687927 	 146.77638114296377
39.269105862946375 	 202.6390244401379
pink
-35.297953544675565 	 -73.54118349264807
-36.19100537009939 	 -19.591158285339805
-34.29214432639872 	 35.350598452500506
-32.60897309597457 	 90.99607542936825
-32.79377784338151 	 147.09516499716634
-29.15387676960839 	 203.087629215594
yellow
-75.73281144172893 	 -72.76709990402198
-77.05516727895217 	 -17.560112396275855
-74.92245945911304 	 37.209957306395935
-72.9220365829947 	 92.16740340091879
-72.78267154550944 	 147.85503088720858
-70.04953867696922 	 203.45936238586083
blue
-674.9584844168133 	 -333.13120083552707
621.4041196580849 	 -466.5040622862201
91.41955395621251 	 -47.94934202218276
88.1908131106472 	 9.101837417541379
93.72235044934817 	 67.38717386310165
-415.4552503850916 	 379.6562049768761
91.05455900182179 	 123.046177